In [1]:
#resample will compensate all empty timestemp
#1h 3h 6h 24h if ih sample 5T

from dotmap import DotMap
from Library.BLRPRmodel.BLRPRx import *
from calendar import month_abbr
from datetime import timedelta as td
from datetime import datetime as dt
from datetime import datetime
from Library.Cal_stats_lib.utils.utils import *
from Library.Cal_stats_lib.utils.stats_calculation import *
import numpy as np
import pandas as pd
import os

from Library.Sampling_lib.mergeCells import *
from Library.Sampling_lib.sampling import *
from Library.Sampling_lib.compare import *
from Library.Fitting_lib import fitting, objectiveFunction
import warnings, yaml

args = yaml.load(open('./config/default.yaml'), Loader=yaml.FullLoader)
args = DotMap(args)
args.sampling.start_time = dt.strptime(args.sampling.start_time, '%Y-%m-%d')
args.sampling.end_time = dt.strptime(args.sampling.end_time, '%Y-%m-%d')

total_sample_sta_prop = []
num_month = pd.read_csv(args.IO.stats_file_path,index_col=0, header=0).shape[0]
pDryThreshold = 0.5
timeseries = []

for month in range(1, num_month+1):
    start_time, end_time, freq = args.sampling.start_time, args.sampling.end_time, args.sampling.freq
    duration_sim_hr = (end_time - start_time).total_seconds() / 3600

    # step 0 read result thetas(lambda, iota, alpha, nu, kappa, phi)
    thetas = pd.read_csv('./02 Output_data/Theta.csv').loc[month-1]
    print('='*50)
    print(f'Mon : {str(month)}/{str(num_month)}')

    # step 1 sample storms
    alpha, nu, phi,storms = SampleStorm(thetas, start_time, duration_sim_hr)

    #import pdb; pdb.set_trace()
    rainfall_ts = MergeCells(storms, freq)
    
    # step 2 calculate the sta prop of the sample ts
    sample_sta_prop = sample_Sta_prop(rainfall_ts, args.IO.config_path, pDryThreshold)
    total_sample_sta_prop.append(sample_sta_prop)
    print("----------Finish row {} sampling!----------\n\n".format(month))

#  step 3 output the sta prop
df = pd.DataFrame(total_sample_sta_prop[0])
for i in range(1, len(total_sample_sta_prop)):
    df2 = pd.DataFrame(total_sample_sta_prop[i])
    fin_df = pd.concat([df, df2])
    df = fin_df

Mon : 1/12
120 samples of storms are generated. 

scale, mData: 1h, 0.07863242939937447
scale, mData: 3h, 0.23581517984859396
scale, mData: 6h, 0.4714662572755813
scale, mData: 1D, 1.881936143625029
----------Finish row 1 sampling!----------


Mon : 2/12
141 samples of storms are generated. 

scale, mData: 1h, 0.07230349360493603
scale, mData: 3h, 0.21686047839876038
scale, mData: 6h, 0.43372095679752076
scale, mData: 1D, 1.7324875788099863
----------Finish row 2 sampling!----------


Mon : 3/12
227 samples of storms are generated. 

scale, mData: 1h, 0.06364618526245856
scale, mData: 3h, 0.19085061632587313
scale, mData: 6h, 0.381569429739919
scale, mData: 1D, 1.526277718959676
----------Finish row 3 sampling!----------


Mon : 4/12
154 samples of storms are generated. 

scale, mData: 1h, 0.0638321113529553
scale, mData: 3h, 0.19140810722907606
scale, mData: 6h, 0.3825518386194379
scale, mData: 1D, 1.5259919072202373
----------Finish row 4 sampling!----------


Mon : 5/12
50 samples o